In [1]:
using PowerSystems
using DataFrames
using Logging
using CSV
gl = global_logger()
LOG_LEVELS = Dict(
    "Debug" => Logging.Debug,
    "Info" => Logging.Info,
    "Warn" => Logging.Warn,
    "Error" => Logging.Error,
)
level = get(ENV, "PS_LOG_LEVEL", "Error")
log_level = get(LOG_LEVELS, level, nothing)
global_logger(ConsoleLogger(gl.stream, log_level));

In [2]:
pm_dict = parse_file("data/ACTIVSg2000.m")
PowerSystems.make_mixed_units(pm_dict)
ps_dict = PowerSystems.pm2ps_dict(pm_dict)

Dict{String,Any} with 9 entries:
  "name"        => "ACTIVSg2000"
  "dcline"      => Dict{String,Any}()
  "baseMVA"     => 100.0
  "gen"         => Dict{String,Any}("Renewable"=>Dict{String,Any}("PV"=>Dict{St…
  "branch"      => Dict{String,Any}("Transformers"=>Dict{String,Any}("2674"=>Di…
  "shunt"       => Dict{String,Any}("32"=>Dict{String,Any}("Y"=>0.0+65.0im,"nam…
  "load"        => Dict{String,Any}("1"=>Dict{String,Any}("name"=>"ODESSA 2 0",…
  "bus"         => Dict{Int64,Any}(1704=>Dict{String,Any}("voltagelimits"=>(min…
  "source_type" => "matpower"

In [3]:
ps_dict["load"]["519"]["maxactivepower"]

23.95

In [4]:
PVdf = DataFrame(bus = [element["bus"].name for element in values(ps_dict["gen"]["Renewable"]["PV"])]);
PVcapdf = [element["tech"]["installedcapacity"] for element in values(ps_dict["gen"]["Renewable"]["PV"])]
PVdf[:Capacity] = PVcapdf;
PVdf[:Source] = "PV";
PVdf
CSV.write("PV_Data.csv", PVdf)

"PV_Data.csv"

In [5]:
Winddf = DataFrame(bus = [element["bus"].name for element in values(ps_dict["gen"]["Renewable"]["WIND"])]);
Windcapdf = [element["tech"]["installedcapacity"] for element in values(ps_dict["gen"]["Renewable"]["WIND"])]
Winddf[:Capacity] = Windcapdf;
Winddf[:Source] = "Wind";
Winddf
CSV.write("Wind_Data.csv", Winddf)

"Wind_Data.csv"

In [6]:
Hydrodf = DataFrame(bus = [element["bus"].name for element in values(ps_dict["gen"]["Hydro"])]);
HydroCapdf = [element["tech"]["installedcapacity"] for element in values(ps_dict["gen"]["Hydro"])]
Hydrodf[:Capacity] = HydroCapdf;
Hydrodf[:Source] = "Hydro";
Hydrodf
CSV.write("Hydro_Data.csv", Hydrodf)

"Hydro_Data.csv"

In [7]:
bus = Vector{String}()
fuel = Vector{String}()
cap = Vector{Float64}()
for (k,v) in ps_dict["gen"]["Thermal"]
    push!(bus, v["bus"].name)
    push!(fuel, pm_dict["gen"][k]["fuel"])
    push!(cap, v["tech"]["activepowerlimits"].max)
end

In [8]:
thermaldf = DataFrame(bus = bus, capacity = cap, fuel = fuel);
thermaldf[:Source] = "Thermal";
thermaldf
CSV.write("Thermal_Data.csv", thermaldf)

"Thermal_Data.csv"

In [9]:
Loaddf = DataFrame(bus = [element["bus"].name for element in values(ps_dict["load"])]);
LoadPower = [element["maxactivepower"] for element in values(ps_dict["load"])]
Loaddf[:PeakPower] = LoadPower;
Loaddf[:Source] = "Load";
Loaddf
CSV.write("Load_Data.csv", Loaddf)

"Load_Data.csv"